In [1]:
from create_data import *

train_path = './marathi_data/UD_Marathi-UFAL/mr_ufal-ud-train.conllu'
dev_path = './marathi_data/UD_Marathi-UFAL/mr_ufal-ud-dev.conllu'
test_path = './marathi_data/UD_Marathi-UFAL/mr_ufal-ud-test.conllu'

wnut, label_list = createData(train_path, dev_path, test_path)
print(label_list)

Shape of train data :  373
Shape of dev data :  46
Shape of test data :  47
Total Unique Tags =  15
['O', 'ADV', 'AUX', 'NOUN', 'PUNCT', 'DET', 'ADJ', 'ADP', 'CCONJ', 'NUM', 'PROPN', 'INTJ', 'PART', 'PRON', 'SCONJ', 'VERB']


In [31]:
wnut

{'train': [{'id': '0',
   'pos_tags': ['DET', 'AUX', 'NOUN', 'PUNCT'],
   'tokens': ['एक', 'असणे', 'राजा', '.']},
  {'id': '1',
   'pos_tags': ['PRON', 'DET', 'NOUN', 'AUX', 'PUNCT'],
   'tokens': ['तो', 'एक', 'मुलगा', 'असणे', '.']},
  {'id': '2',
   'pos_tags': ['ADJ',
    'DET',
    'NOUN',
    'CCONJ',
    'NOUN',
    'PUNCT',
    'NOUN',
    'PRON',
    'NOUN',
    'ADV',
    'NOUN',
    'VERB',
    'PUNCT'],
   'tokens': ['एकुलता',
    'एक',
    'मुलगा',
    'म्हणून',
    'राजा',
    '-',
    'राणी',
    'तो',
    'जीव',
    'की',
    'प्राण',
    'करित',
    '.']},
  {'id': '3',
   'pos_tags': ['CCONJ', None, 'NOUN', 'ADP', 'PRON', 'VERB', 'PUNCT'],
   'tokens': ['परंतु', None, 'लाड', 'मुळे', 'तो', 'बिघडणे', '.']},
  {'id': '4',
   'pos_tags': ['NOUN',
    'NOUN',
    'VERB',
    'PUNCT',
    'PUNCT',
    'PRON',
    'VERB',
    'VERB',
    'PUNCT',
    'PUNCT'],
   'tokens': ['राजा',
    'मन',
    'म्हणणे',
    ',',
    ',',
    'हा',
    'घालवणे',
    'देणे',
    '.',
    '"']}

In [10]:
import spacy

text = "My name is Itisha. I like playing music and I am very cute."
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
for tok in doc:
    print(tok.text, tok.pos_)

My PRON
name NOUN
is AUX
Itisha PROPN
. PUNCT
I PRON
like VERB
playing VERB
music NOUN
and CCONJ
I PRON
am AUX
very ADV
cute ADJ
. PUNCT


In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")


In [30]:
tokenized_input = tokenizer(wnut["train"][0]["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokenized_input.word_ids(batch_index=0)

[None, 0, 1, 1, 1, 2, 3, None]

In [32]:
def tokenize_and_align_labels(examples):
    print(examples)
    print(examples["tokens"])
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    print(tokenized_inputs)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
    print(tokens)
    print()
    labels = []
    for i, label in enumerate(examples[f"pos_tags"]):
        print(i, label)
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        print(word_ids)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:                            # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                # Only label the first token of a given word.
                print(word_idx)
                label_ids.append(label[word_idx])
            #print(label_ids, label)

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenize_and_align_labels(wnut["train"][0])

{'id': '0', 'pos_tags': ['DET', 'AUX', 'NOUN', 'PUNCT'], 'tokens': ['एक', 'असणे', 'राजा', '.']}
['एक', 'असणे', 'राजा', '.']
{'input_ids': [101, 11186, 851, 13432, 30616, 36331, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'एक', 'अ', '##स', '##णे', 'राजा', '.', '[SEP]']

0 DET
[None, 0, 1, 1, 1, 2, 3, None]
0
1
1
1
2
3


IndexError: string index out of range

In [4]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
model = AutoModelForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels=len(label_list)),

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 